# prospect + specutils

Project: Convert DESI, SDSS & eBOSS spectra to specutils container objects, and modify prospect to accept those objects. Try not to use any DESI software besides Prospect itself.

In [ ]:
import os
import sys
# sys.path.insert(0, os.path.join(os.environ['HOME'], 'Documents', 'Code', 'git', 'desihub', 'desiutil', 'py'))
# sys.path.insert(0, os.path.join(os.environ['HOME'], 'Documents', 'Code', 'git', 'desihub', 'desitarget', 'py'))
sys.path.insert(0, os.path.join(os.environ['HOME'], 'Documents', 'Code', 'git', 'desihub', 'prospect', 'py'))

In [ ]:
# import os
import numpy as np
from astropy.table import Table
from specutils import Spectrum1D, SpectrumCollection, SpectrumList
from prospect.specutils import read_spectra, read_spPlate, read_spZbest
from prospect.viewer import plotspectra

## DESI spectra file

DESI spectra are stored by "channel" ("channel", "band" and "spectrograph arm" are used interchangably).  There are 10 spectrographs, each with three arms, called 'b', 'r' & 'z'.  In a DESI spectra file, all arms are grouped together, with a common wavelength solution for each arm, but the solutions do not overlap or have the same shape.  Thus we can't use a Spectrum1D or SpectrumCollection object, but we can use a SpectrumList containing three Spectrum1D objects.

In [ ]:
os.environ['DESI_ROOT'] = '/global/cfs/cdirs/desi'
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'spectro', 'redux')
os.environ['SPECPROD'] = 'iron'
survey = 'main'
program = 'dark'
pixnum = '5350'
pixgroup = pixnum[0:2]
desi_spectra = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'], 'healpix', survey, program, pixgroup, pixnum, f'coadd-{survey}-{program}-{pixnum}.fits')
desi_redshifts = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'], 'healpix', survey, program, pixgroup, pixnum, f'redrock-{survey}-{program}-{pixnum}.fits')
print(desi_spectra)
print(desi_redshifts)

## SDSS/eBOSS spPlate file

SDSS spectra are stored per-plate in spPlate files.  These contain 640 spectra for the original SDSS spectrograph or 1000 spectra for the BOSS/eBOSS spectrograph.  All spPlate files have a common wavelength solution, so a spPlate file can be represented by a Spectrum1D object.

In [ ]:
os.environ['SDSS_ROOT'] = '/global/cfs/cdirs/sdss'
os.environ['SPECTRO_REDUX'] = os.path.join(os.environ['SDSS_ROOT'], 'data', 'sdss', 'dr17', 'sdss', 'spectro', 'redux')
run2d = '26'
plate = '2955'
mjd = '54562'
sdss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
sdss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spZbest-{plate}-{mjd}.fits')
print(sdss_spectra)
print(sdss_redshifts)

In [ ]:
run2d = 'v5_13_2'
plate = '9599'
mjd = '58131'
eboss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
eboss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, run2d, f'spZbest-{plate}-{mjd}.fits')
print(eboss_spectra)
print(eboss_redshifts)

## DESI spectra

Read the spectra and corresponding wavelength solutions.  Multiple exposures for each spectra may exist, so we do a simple coadd to get one spectrum per object.  We'll need to clean up some anomalous values first.

In [ ]:
desi_z = Table.read(desi_redshifts, 'REDSHIFTS')
w = np.where(desi_z['TARGETID'] > 0)[0][0:50]
desi_z = desi_z[w]
# Since the spectra are already coadded, this is equivalent to limiting the number of spectra to len(desi_z['TARGETID']).
desi = read_spectra(desi_spectra, single=True, coadd=desi_z['TARGETID'])
desi

### TO DO

Create model spectra without using redrock software.

In [ ]:
model_spectral_axis = np.linspace(desi[0].spectral_axis[0].value, desi[2].spectral_axis[-1].value, num=7000)
model_flux = np.zeros((desi[0].flux.shape[0], 7000), dtype=desi[0].flux.dtype)

In [ ]:
foo, bar = plotspectra(desi, zcatalog=desi_z, redrock_cat=None, notebook=True, title=os.path.basename(desi_spectra),
                       with_thumb_tab=False, with_vi_widgets=False, with_coaddcam=False, mask_type=None,
                       model_from_zcat=False, model=(model_spectral_axis, model_flux),
                       with_full_2ndfit=False)

## SDSS Spectra

In [ ]:
sdss = read_spPlate(sdss_spectra, limit=50)
sdss_z, sdss_model = read_spZbest(sdss_redshifts, limit=50)
sdss

In [ ]:
foo, bar = plotspectra(sdss, zcatalog=sdss_z, notebook=True, title=os.path.basename(sdss_spectra),
                       with_thumb_tab=False, with_vi_widgets=False, with_coaddcam=False, mask_type=None,
                       model_from_zcat=False, model=(sdss_model.spectral_axis.value, sdss_model.flux.value),
                       with_full_2ndfit=False)

## eBOSS Spectra

In [ ]:
eboss = read_spPlate(eboss_spectra, limit=50)
eboss_z, eboss_model = read_spZbest(eboss_redshifts, limit=50)
eboss

In [ ]:
foo, bar = plotspectra(eboss, zcatalog=eboss_z, notebook=True, title=os.path.basename(eboss_spectra),
                       with_thumb_tab=False, with_vi_widgets=False, with_coaddcam=False, mask_type=None,
                       model_from_zcat=False, model=(eboss_model.spectral_axis.value, eboss_model.flux.value),
                       with_full_2ndfit=False)